In [ ]:
import os
# os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.7/bin") # path to nvidia dlls, must have CUDA drivers installed
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import mixed_precision

### GPU Acceleration

** IMPORTANT NOTE **
 - You must have CUDA driver and CUDA toolkit installed on your machine. To do so, follow the instructions here:
https://www.tensorflow.org/install/pip

- Enable GPU Acceleration if you have a NVIDIA GPU with compute capability >= 7.0

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

# ## ENABLE THIS IF YOU HAVE RTX GPU WITH COMPUTE CAPABILITY 7.0 or higher
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_global_policy(policy)
# print('Compute dtype: %s' % policy.compute_dtype)
# print('Variable dtype: %s' % policy.variable_dtype)

### Loading Data

In [ ]:
fp = "../data/features_combined.csv"
batch_pd = pd.read_csv(fp, index_col=False)
dataset = batch_pd.copy()
dataset.sort_values(by=['policy'], ascending=True, inplace=True)

# dataset
dataset.isna().sum()
dataset = dataset.dropna().drop(columns=['policy', 'barcode'])

### Test run setup
1. Adjust global model config to test the model settings
2. Change the Run configuration to define the saved location for the results and the number of runs
3. Run the notebook
4. Clear output and restart the notebook for the next run.

In [ ]:
## Global Model Config
EPOCHS = 2500 # number of epochs to train the model
UNITS = 1 # number of units/nodes in the hidden layer
LEARNING_RATE = 0.01 # learning rate for the model
CALLBACK = tf.keras.callbacks.EarlyStopping(monitor='RMSE', patience=15, min_delta=0.01)


#### Run configuration

In [ ]:
## Define the run setup

RUN_NUMBER = 1 # Test run number
SAVED_RESULT_PATH = "../variance_model/results/test_run#" + str(RUN_NUMBER) # path to saved results

if ( os.path.exists(SAVED_RESULT_PATH) ):
    print("Directory " , SAVED_RESULT_PATH , " already exists")
else:
    os.mkdir(SAVED_RESULT_PATH)
    print("Directory " , SAVED_RESULT_PATH , " created")

## Data split
Split by policy fast charge first(5C - 8C), and then by policy slow charge (1C - 4C)


In [ ]:
normal_charge_dataset = dataset.iloc[0:29, :] # 29 data points
print(normal_charge_dataset.shape)

## Normal-Charge Test-Train split
Selecting alternate batteries for training and testing

In [ ]:
normal_charge_train_ds = normal_charge_dataset.iloc[0::2, :]
normal_charge_test_ds = normal_charge_dataset.iloc[1::2, :]

normalcharge_train_features = normal_charge_train_ds.copy()
normalcharge_test_features = normal_charge_test_ds.copy()

normal_train_labels = normalcharge_train_features.pop('cycle_life')
normal_test_labels = normalcharge_test_features.pop('cycle_life')

# Linear Regress
### Layering and Build Model

In [ ]:
# Normalize layer
QDiffLinVar = np.array(normalcharge_train_features['QDiffLinVar'])
QDiffLinVar_normalizer = layers.Normalization(input_shape=[1,], axis=None)
QDiffLinVar_normalizer.adapt(QDiffLinVar)

variance_normalcharge_basemodel = tf.keras.Sequential([
    QDiffLinVar_normalizer,
    layers.Dense(UNITS, input_dim=1, activation='relu'),
    layers.Dense(1, activation='linear', dtype='float32', name='predictions')
])
variance_normalcharge_basemodel.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[ 
        tf.keras.metrics.RootMeanSquaredError( name='RMSE'), 
        'mae']
    )

variance_normalcharge_basemodel.summary()

print("Number of weights weights:", len(variance_normalcharge_basemodel.weights))
print("trainable_weights:", len(variance_normalcharge_basemodel.trainable_weights))
print("non_trainable_weights:", len(variance_normalcharge_basemodel.non_trainable_weights))


### Train the Model

In [ ]:
%%time
history = variance_normalcharge_basemodel.fit(
    normalcharge_train_features['QDiffLinVar'],
    normal_train_labels,
    epochs=EPOCHS,
    verbose=2,
    callbacks=[CALLBACK],
    validation_data=(normalcharge_test_features['QDiffLinVar'], normal_test_labels))

### Plot loss graph and Evaluate

In [ ]:
def plot_loss(basemodel_hist):
  plt.figure("Normal-charge base model loss", figsize=(5,5), dpi=100, facecolor='w', edgecolor='k')
  plt.plot(basemodel_hist['RMSE'], label='loss_base', color='red')
  plt.plot(basemodel_hist['val_RMSE'], label='val_loss_base', color='green', linestyle='dashed')
  plt.ylim([0, 800])
  plt.xlabel('Epoch')
  plt.ylabel('Error [cycles]')
  plt.legend()
  plt.grid(True)
  plt.title('Normal-charge base model loss')
  plt.savefig(SAVED_RESULT_PATH + "/normalcharge_basemodel_loss_RUN#" + str(RUN_NUMBER) + ".png")

## Display model's loss and accuracy history
basemodel_history = pd.DataFrame(history.history)
basemodel_history.to_csv(SAVED_RESULT_PATH + "/normalcharge_basemodel_history_RUN#" + str(RUN_NUMBER) + ".csv")
basemodel_history

plot_loss(basemodel_history)

In [ ]:
test_results = {}
test_results['variance_normalcharge_basemodel'] = variance_normalcharge_basemodel.evaluate(
    normalcharge_test_features['QDiffLinVar'],
    normal_test_labels, verbose=1) #sqrt for mse

### Make Predictions

In [ ]:
def plot_prediction(y_train, y_test):
  plt.figure("Normal-charge base model loss", figsize=(5,5), dpi=100, facecolor='w', edgecolor='k')
  plt.axes(aspect='equal')
  plt.scatter(y_train, normal_train_labels, label='Predictions (train)')
  plt.scatter(y_test, normal_test_labels, label='Predictions (test)', marker='^')
  lims = [0, 2000]
  plt.xlim(lims)
  plt.ylim(lims)
  plt.plot(lims, lims, 'k', )
  plt.xlabel('Predicted Cycle life')
  plt.ylabel('Actual Cycle life')
  plt.legend()
  plt.title('Normal-charge Model Prediction')
  plt.savefig( SAVED_RESULT_PATH + '/normalcharge_basemodel_prediction_RUN#' + str(RUN_NUMBER) + ".png")

normal_train_prediction = variance_normalcharge_basemodel.predict(normal_charge_train_ds['QDiffLinVar'])
normal_test_prediction = variance_normalcharge_basemodel.predict(normal_charge_test_ds['QDiffLinVar'])
plot_prediction(normal_train_prediction, normal_test_prediction)

### Final Results

In [ ]:
result = pd.DataFrame(test_results, index=['MSE', 'RMSE', 'MAE']).T
result.to_csv( SAVED_RESULT_PATH + '/normalcharge_basemodel_result_RUN#' + str(RUN_NUMBER) + ".csv")

### Saving the model

In [ ]:
variance_normalcharge_basemodel.save('../variance_model/saved_model/variance_normalcharge_basemodel')